In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

min_buy_val = 20
max_buy_val = 100
num_days_lin_reg = 20

In [ ]:
df = pd.read_excel('all stocks50 days.xlsx', index_col=0)

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['Capital Gains'])
df_copy = df_copy.drop(columns=['Adj Close'])
df_copy = df_copy[df_copy['Volume'] > 0]
df_copy = df_copy[(df_copy['Open'] >= min_buy_val) & (df_copy['Open'] <= max_buy_val) ]

#display(df_copy)

In [ ]:
indexes = df_copy.Symbol.value_counts().loc[lambda x: x>=50].index  #get indexes that have 50 or more entries
df_g50 = df_copy[df_copy['Symbol'].isin(indexes)] #put only those indexes into df
df_g50 = df_g50.reset_index()
df_g50 = df_g50.drop(columns=['Date'])

#display(df_g50)

In [ ]:
#this cell runs slow - i will buy you a cookie if you can speed it up

reg = LinearRegression()
df_g50['Open slope'] = 0
df_g50['Open Linear reg MSE'] = 0

X_train = np.arange(0, 20, 1).reshape(-1, 1)

unique_symbols = df_g50['Symbol'].unique()

#for index in range(5):   #for testing - leave in place
 #   symbol = unique_symbols[index]

#predict on the first 20 and test on the same - in future versions these will be learned params
for symbol in unique_symbols:
    subset = df_g50[df_g50['Symbol'] == symbol].iloc[0:num_days_lin_reg, :]
    Y1 = subset['Open']
    
    model1 = reg.fit(X_train, Y1)
    prediction1 = model1.predict(X_train)
    mse1 = mean_squared_error(Y1, prediction1)
    
    subset = df_g50[df_g50['Symbol'] == symbol]
    df_g50.loc[subset.index, 'Open slope'] = model1.coef_[0]
    df_g50.loc[subset.index, 'Open Linear reg MSE'] = mse1

stocks_to_buy = df_g50.groupby('Symbol').apply(lambda group: group.iloc[20:, :])

In [ ]:
display(stocks_to_buy)
stocks_to_buy.to_csv('stocks to buy.csv', index=True)